In [34]:
import numpy as np
import pandas as pd
import os
import torch
import torch.optim
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.preprocessing import OneHotEncoder
from torch.autograd import Variable
import cv2

In [23]:
data = pd.read_csv('styles.csv')

ID = data['id'].values    #图片的名称


label = data['articleType'].values

#label = tf.one_hot(label,depth=12)
a = []
for i in label:
    a.append(i)
label = np.array(a)
one_hot = OneHotEncoder()
label = one_hot.fit_transform(label.reshape(-1,1)).toarray()   #进行one_hot编码。
print(label[:10])
print(label.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
(44440, 143)


In [36]:
#重定义Dataset
class DataSet(Dataset):
    def __init__(self):
        self.path = [os.path.join(".\images\\",str(i)+".jpg") for i in ID]
    
    def __getitem__(self,index):
        path = self.path[index]
        img = cv2.imread(path)
        img = cv2.resize(img,(128,128))
        img = img.reshape(-1,128,128)
        img = torch.from_numpy(img).type(torch.float32)
        y_true = np.argmax(label[index],-1).astype(np.longlong)   #交叉熵损失函数不能使用one-hot编码，而且数据要是long类型的
        
        return img,y_true
    def __len__(self):
        return len(self.path)
    
dataset = DataSet()
data_loader = DataLoader(dataset,batch_size=128,shuffle=True,drop_last=True)
next(iter(dataset))

(tensor([[[255., 255., 255.,  ..., 255., 255., 254.],
          [255., 255., 254.,  ..., 254., 255., 254.],
          [254., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [122., 118., 160.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 106., 143., 134.],
          [133., 180., 169.,  ...,  94.,  88., 132.]],
 
         [[120., 116., 152.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ...,  75., 106.,  96.],
          [ 95., 136., 125.,  ...,  72.,  66., 123.],
          ...,
          [135., 146., 131.,  ..., 127., 125., 133.],
          [120., 119., 129.,  ..., 251., 254., 255.],
          [255., 255., 247.,  ..., 127., 127., 114.]],
 
         [[120., 123., 108.,  ..., 138., 139., 151.],
          [138., 139., 152.,  ..., 251., 254., 255.],
          [255., 255., 246.,  ..., 134., 128., 115.],
          ...,
          [248., 255., 255.,  ...,  72., 111.,  97.],
          [ 76., 117., 103.,  ...,  74.,  45.,  68.],
          [ 61.,  38.,  68.,  .

In [37]:
#定义网络
class Net_work(nn.Module):
    def __init__(self):
        super(Net_work, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3,stride=1,padding=1),
            nn.MaxPool2d(stride=2,kernel_size=2),     # 64 64
            nn.BatchNorm2d(16),
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16,64,kernel_size=3,stride=1,padding=1),
            nn.MaxPool2d(stride=2,kernel_size=2),  # 32 32
            nn.BatchNorm2d(64),
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.MaxPool2d(stride=2,kernel_size=2),   #16 16
            nn.BatchNorm2d(128)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(128*16*16,2048),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.2),
            nn.Linear(2048,256),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.2),
            nn.Linear(256,143),
            nn.Softmax(dim=-1)
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = x.view(x.size(0),-1)

        x = self.fc1(x)

        return x

net_work = Net_work()

In [40]:
#定义损失函数和优化函数
optimizer = torch.optim.Adam(net_work.parameters(),lr=0.005)
error = nn.CrossEntropyLoss()
net_work.train()    #设置为训练模式

Net_work(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Sequential(
    (0): Linear(in_features=32768, out_features=2048, bias=True)
    (1): Dropout(p=0.5)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Lin

In [ ]:
#开始训练
for i in range(2):
    for x,y in data_loader:
        x = Variable(x)

        y = Variable(y)
        optimizer.zero_grad()

        pred = net_work(x)

        loss = error(pred,y)

        loss.backward()

        optimizer.step()

        print(loss)

tensor(4.8498, grad_fn=<NllLossBackward>)
tensor(4.7852, grad_fn=<NllLossBackward>)


In [ ]:
torch.save(net.state_dict(),"a.pth")   #训练完成后保存模型